In [28]:
!pip install -q langchain sentence-transformers faiss-cpu "transformers[torch]" langchain-community

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:

import os
from google.colab import drive

print("📁 Mounting Google Drive...")
drive.mount('/content/drive')


PROJECT_DIR = '/content/drive/MyDrive/AI_Assignment'
STAGE1_OUTPUT_DIR = f"{PROJECT_DIR}/stage1_outputs"
STAGE2_DIR = f"{PROJECT_DIR}/stage2_outputs"


os.makedirs(STAGE2_DIR, exist_ok=True)

print(f"✅ Project directory: {PROJECT_DIR}")
print(f"✅ Stage 1 outputs: {STAGE1_OUTPUT_DIR}")
print(f"✅ Stage 2 outputs: {STAGE2_DIR}")
print("\n📂 Directory structure ready!")

📁 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Project directory: /content/drive/MyDrive/AI_Assignment
✅ Stage 1 outputs: /content/drive/MyDrive/AI_Assignment/stage1_outputs
✅ Stage 2 outputs: /content/drive/MyDrive/AI_Assignment/stage2_outputs

📂 Directory structure ready!


In [ ]:

import torch
import pandas as pd
import numpy as np
import json
from datetime import datetime
from transformers import pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate


np.random.seed(42)
torch.manual_seed(42)


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️  Using device: {device}")
if device == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("   ⚠️  WARNING: GPU not detected!")
    print("   Go to: Runtime → Change runtime type → Hardware accelerator → T4 GPU")
    print("   Then: Runtime → Restart runtime")

print("\n✅ All libraries imported successfully!")

🖥️  Using device: cuda
   GPU: Tesla T4
   Memory: 15.83 GB

✅ All libraries imported successfully!


In [ ]:

import os
stage1_dir = '/content/drive/MyDrive/AI_Assignment/stage1_outputs'
if os.path.exists(stage1_dir):
    print("Files in stage1_outputs:")
    for f in os.listdir(stage1_dir):
        print(f"  - {f}")
else:
    print("Folder doesn't exist!")
    print("\nChecking parent folder:")
    parent = '/content/drive/MyDrive/AI_Assignment'
    if os.path.exists(parent):
        print("Files in AI_Assignment:")
        for f in os.listdir(parent):
            print(f"  - {f}")

Files in stage1_outputs:
  - training_metrics.txt
  - evaluation_metrics.txt
  - stage1_predictions.csv
  - confusion_matrix.png
  - gradcam_img_0_true_anger.png
  - gradcam_img_1_true_surprise.png
  - gradcam_img_2_true_neutral.png
  - gradcam_img_3_true_sadness.png


In [ ]:

print("="*60)
print("TASK 1: DATA GENERATION")
print("="*60)

predictions_path = f"{STAGE1_OUTPUT_DIR}/stage1_predictions.csv"
print(f"\n📂 Loading predictions from: {predictions_path}")

try:
    pred_df = pd.read_csv(predictions_path)
    print(f"✅ Loaded {len(pred_df)} predictions")
    print(f"\n📊 Columns: {list(pred_df.columns)}")
    print(f"\n🔍 First few rows:")
    print(pred_df.head())

    
    if len(pred_df) > 500:
        sample_df = pred_df.sample(n=500, random_state=42)
        print(f"\n✂️  Sampled 500 predictions for Stage 2")
    else:
        sample_df = pred_df
        print(f"\n📝 Using all {len(sample_df)} predictions")

except FileNotFoundError:
    print(f"❌ ERROR: Cannot find predictions file at {predictions_path}")
    print("\nPlease ensure Stage 1 is complete and the file exists.")
    raise


print("\n📝 Defining review templates...")
templates = {
    'anger': [
        "Absolutely furious with purchase quality issues.",
        "Worst customer service experience ever encountered.",
        "Completely dissatisfied, demanding immediate refund now.",
        "Outrageous product defects, unacceptable standards.",
        "This made me livid, terrible quality control.",
        "Infuriating delivery delays and poor communication.",
        "Rage-inducing false advertising and broken promises.",
        "Extremely upset, product broke within days.",
        "Beyond angry, company refuses responsibility.",
        "Angry doesn't describe my frustration adequately."
    ],
    'happiness': [
        "Absolutely thrilled with exceptional product quality!",
        "Best purchase decision made this year!",
        "Exceeded expectations in every possible way!",
        "Delighted by outstanding customer service experience!",
        "Perfect product, brings daily joy and satisfaction!",
        "Highly recommend, worth every single penny!",
        "Love everything, couldn't ask for better!",
        "Ecstatic about fast shipping and quality!",
        "Overjoyed with results, exactly as advertised!",
        "Happiest customer, will definitely buy again!"
    ],
    'sadness': [
        "Disappointed by poor quality, money wasted.",
        "Had hopes but product underperformed significantly.",
        "Really sad experience, failed expectations completely.",
        "Heartbroken by misleading product descriptions online.",
        "Feeling let down, not worth price paid.",
        "Melancholy purchase, wish never bought it.",
        "Down about experience, expected better results.",
        "Dejected by lack of support after sale.",
        "Sorrowful decision, complete buyer's remorse now.",
        "Disappointed tears, absolute waste of money."
    ],
    'surprise': [
        "Wow, totally unexpected excellent performance results!",
        "Genuinely shocked by superior quality levels!",
        "Surprising excellence, never anticipated this!",
        "Astonishing value, exceeded wildest dreams!",
        "Caught off guard by phenomenal features!",
        "What shock, blown away completely!",
        "Unexpected brilliance in design and function!",
        "Startling effectiveness, pleasantly amazed daily!",
        "Incredible surprise beyond initial expectations!",
        "Jaw-dropping quality, never saw coming!"
    ],
    'fear': [
        "Worried about serious safety concerns here.",
        "Frighteningly poor build quality risks.",
        "Nerve-wracking unreliability causes daily stress.",
        "Genuinely concerned about potential hazards.",
        "Terrifying defects could cause injuries.",
        "Anxious using product, seems dangerous.",
        "Dreadful quality, fearful of failures.",
        "Alarming issues, scared it will break.",
        "Intimidating risks, worried about consequences.",
        "Frightened by obvious manufacturing flaws."
    ],
    'disgust': [
        "Disgusting lack of basic quality standards.",
        "Repulsed by terrible craftsmanship throughout.",
        "Awful condition, completely unacceptable filth.",
        "Revolting product arrived damaged and dirty.",
        "Absolutely repellent, can't believe standards.",
        "Nauseating poor quality offends customers.",
        "Repugnant disregard for basic standards.",
        "Sickening experience from beginning to end.",
        "Abhorrent quality control completely absent.",
        "Vile product, utterly disgusted entirely."
    ],
    'neutral': [
        "Product performs adequately for basic needs.",
        "Average quality, meets minimum expectations only.",
        "Neutral opinion, neither impressed nor disappointed.",
        "Standard functionality, works as described.",
        "Decent purchase, gets job done sufficiently.",
        "Acceptable quality level for price paid.",
        "Mediocre experience overall, nothing special.",
        "Satisfactory but unremarkable in all aspects.",
        "Ordinary quality, functions properly enough.",
        "Adequate product serving intended purpose."
    ]
}

print(f"✅ Loaded {len(templates)} emotion categories with {sum(len(v) for v in templates.values())} total templates")


print(f"\n🔄 Generating synthetic reviews...")
reviews_docs = []
review_data = []
used_combinations = set() 

for idx, row in sample_df.iterrows():
    emotion = row['predicted_label_name']

   
    max_attempts = 10
    for attempt in range(max_attempts):
        template_idx = np.random.randint(0, len(templates[emotion]))
        combo = (emotion, template_idx)
        if combo not in used_combinations or attempt == max_attempts - 1:
            used_combinations.add(combo)
            break

    review_text = templates[emotion][template_idx]

   
    doc = Document(
        page_content=review_text,
        metadata={
            "emotion": emotion,
            "true_emotion": row['true_label_name']
        }
    )
    reviews_docs.append(doc)

    
    review_data.append({
        'predicted_emotion': emotion,
        'true_emotion': row['true_label_name'],
        'review_text': review_text
    })

print(f"✅ Successfully generated {len(reviews_docs)} synthetic reviews")
print(f"   Unique combinations used: {len(used_combinations)}")


reviews_csv_path = f"{STAGE2_DIR}/generated_reviews.csv"
reviews_df = pd.DataFrame(review_data)
reviews_df.to_csv(reviews_csv_path, index=False)
print(f"💾 Saved reviews to: {reviews_csv_path}")


print(f"\n📊 Emotion Distribution in Generated Reviews:")
emotion_counts = reviews_df['predicted_emotion'].value_counts()
for emotion, count in emotion_counts.items():
    print(f"   {emotion}: {count} ({count/len(reviews_df)*100:.1f}%)")


print(f"\n📝 Sample Generated Reviews:")
for emotion in list(templates.keys())[:3]:
    emotion_samples = reviews_df[reviews_df['predicted_emotion'] == emotion]
    if len(emotion_samples) > 0:
        sample = emotion_samples.iloc[0]
        print(f"\n  [{emotion.upper()}]")
        print(f"  \"{sample['review_text']}\"")
    else:
        print(f"\n  [{emotion.upper()}]")
        print(f"  (No samples in this batch)")

print("\n" + "="*60)
print("✅ TASK 1 COMPLETE: Data Generation")
print("="*60)

TASK 1: DATA GENERATION

📂 Loading predictions from: /content/drive/MyDrive/AI_Assignment/stage1_outputs/stage1_predictions.csv
✅ Loaded 3589 predictions

📊 Columns: ['true_label_id', 'predicted_label_id', 'true_label_name', 'predicted_label_name']

🔍 First few rows:
   true_label_id  predicted_label_id true_label_name predicted_label_name
0              0                   0           anger                anger
1              5                   4        surprise              sadness
2              6                   2         neutral                 fear
3              4                   4         sadness              sadness
4              2                   4            fear              sadness

✂️  Sampled 500 predictions for Stage 2

📝 Defining review templates...
✅ Loaded 7 emotion categories with 70 total templates

🔄 Generating synthetic reviews...
✅ Successfully generated 500 synthetic reviews
   Unique combinations used: 60
💾 Saved reviews to: /content/drive/MyDrive/AI_A

In [ ]:

print("="*60)
print("TASK 2: EMBEDDING & VECTOR STORE SETUP")
print("="*60)


print("\n🔄 Loading sentence-transformer embedding model...")
print("   Model: sentence-transformers/all-MiniLM-L6-v2")
print(f"   Device: {device}")

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True}
)

print("✅ Embedding model loaded successfully")


print(f"\n🔄 Creating FAISS vector database from {len(reviews_docs)} documents...")
print("   This will embed all reviews using the GPU...")

import time
start_time = time.time()

db = FAISS.from_documents(reviews_docs, embeddings)

elapsed = time.time() - start_time
print(f"✅ Vector database created in {elapsed:.2f} seconds")
print(f"   Embedding speed: {len(reviews_docs)/elapsed:.1f} documents/second")


faiss_index_path = f"{STAGE2_DIR}/faiss_index"
print(f"\n💾 Saving FAISS index to: {faiss_index_path}")
db.save_local(faiss_index_path)
print("✅ FAISS index saved successfully")


print("\n🔍 Testing similarity search...")
test_query = "Tell me about angry customers"
results = db.similarity_search(test_query, k=3)

print(f"\n   Query: \"{test_query}\"")
print(f"   Top 3 Results:")
for i, doc in enumerate(results, 1):
    print(f"\n   {i}. [{doc.metadata['emotion'].upper()}]")
    print(f"      \"{doc.page_content[:100]}...\"")

print("\n" + "="*60)
print("✅ TASK 2 COMPLETE: Embedding & Vector Store Setup")
print("="*60)

TASK 2: EMBEDDING & VECTOR STORE SETUP

🔄 Loading sentence-transformer embedding model...
   Model: sentence-transformers/all-MiniLM-L6-v2
   Device: cuda
✅ Embedding model loaded successfully

🔄 Creating FAISS vector database from 500 documents...
   This will embed all reviews using the GPU...
✅ Vector database created in 0.19 seconds
   Embedding speed: 2696.6 documents/second

💾 Saving FAISS index to: /content/drive/MyDrive/AI_Assignment/stage2_outputs/faiss_index
✅ FAISS index saved successfully

🔍 Testing similarity search...

   Query: "Tell me about angry customers"
   Top 3 Results:

   1. [ANGER]
      "Rage-inducing false advertising and broken promises...."

   2. [ANGER]
      "Rage-inducing false advertising and broken promises...."

   3. [ANGER]
      "Rage-inducing false advertising and broken promises...."

✅ TASK 2 COMPLETE: Embedding & Vector Store Setup


In [ ]:

print("="*60)
print("TASK 3: RAG QUERY SYSTEM USING LANGCHAIN")
print("="*60)


print("\n🔄 Loading open-source LLM for RAG pipeline...")
print("   Model: google/flan-t5-base")
print(f"   Device: {device}")

llm_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_length=150,
    do_sample=True,
    temperature=0.3,
    top_p=0.9,
    device=0 if device == "cuda" else -1
)

from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=llm_pipeline)
print("✅ LLM loaded successfully")

# Create improved prompt template
print("\n📝 Creating custom prompt template...")
prompt_template = """Based on the customer reviews below, provide a clear and concise summary that directly answers the question.

Customer Reviews:
{context}

Question: {question}

Provide a brief summary (2-3 sentences maximum):"""

from langchain.prompts import PromptTemplate
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)


from langchain.schema.retriever import BaseRetriever
from langchain.callbacks.manager import CallbackManagerForRetrieverRun
from typing import List

class DiverseRetriever(BaseRetriever):
    vectorstore: object
    k: int = 4

    class Config:
        arbitrary_types_allowed = True

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun = None
    ) -> List:
     
        candidates = self.vectorstore.similarity_search(query, k=20)

       
        selected = []
        seen_texts = set()

        for doc in candidates:
            text_start = doc.page_content[:50]  # First 50 chars
            if text_start not in seen_texts and len(selected) < self.k:
                selected.append(doc)
                seen_texts.add(text_start)

       
        if len(selected) < self.k:
            for doc in candidates:
                if len(selected) >= self.k:
                    break
                if doc not in selected:
                    selected.append(doc)

        return selected[:self.k]


print("\n🔗 Building RAG chain with custom diverse retriever...")
diverse_retriever = DiverseRetriever(vectorstore=db, k=4)

from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=diverse_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

print("✅ RAG pipeline is ready!")
print(f"   Retriever: Custom diverse retriever (k=4, forced uniqueness)")
print(f"   LLM: google/flan-t5-base")
print(f"   Chain type: Stuff (all docs in context)")

print("\n" + "="*60)
print("✅ TASK 3 COMPLETE: RAG Query System Created")
print("="*60)

TASK 3: RAG QUERY SYSTEM USING LANGCHAIN

🔄 Loading open-source LLM for RAG pipeline...
   Model: google/flan-t5-base
   Device: cuda


Device set to use cuda:0


✅ LLM loaded successfully

📝 Creating custom prompt template...

🔗 Building RAG chain with custom diverse retriever...
✅ RAG pipeline is ready!
   Retriever: Custom diverse retriever (k=4, forced uniqueness)
   LLM: google/flan-t5-base
   Chain type: Stuff (all docs in context)

✅ TASK 3 COMPLETE: RAG Query System Created


In [ ]:

print("="*60)
print("TASK 4: SENTIMENT ANALYSIS INTEGRATION")
print("="*60)


print("\n🔄 Loading 3-label sentiment analysis model...")
print("   Model: cardiffnlp/twitter-roberta-base-sentiment-latest")
print(f"   Device: {device}")

sentiment_pipe = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    device=0 if device == "cuda" else -1
)


sentiment_labels = {
    "LABEL_0": "NEGATIVE",
    "LABEL_1": "NEUTRAL",
    "LABEL_2": "POSITIVE"
}

print("✅ Sentiment analysis model loaded successfully")


print("\n🔍 Testing sentiment analysis...")
test_texts = [
    "I am so happy with this purchase!",
    "This is absolutely terrible and disgusting.",
    "The product is fine, nothing special."  
]

for text in test_texts:
    sentiment = sentiment_pipe(text)[0]
    label_name = sentiment_labels.get(sentiment['label'], sentiment['label'])

    print(f"\n   Text: \"{text}\"")
    print(f"   Sentiment: {label_name} (confidence: {sentiment['score']:.3f})")

print("\n" + "="*60)
print("✅ TASK 4 COMPLETE: Sentiment Analysis Integration")
print("="*60)

TASK 4: SENTIMENT ANALYSIS INTEGRATION

🔄 Loading 3-label sentiment analysis model...
   Model: cardiffnlp/twitter-roberta-base-sentiment-latest
   Device: cuda


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


✅ Sentiment analysis model loaded successfully

🔍 Testing sentiment analysis...

   Text: "I am so happy with this purchase!"
   Sentiment: positive (confidence: 0.989)

   Text: "This is absolutely terrible and disgusting."
   Sentiment: negative (confidence: 0.938)

   Text: "The product is fine, nothing special."
   Sentiment: neutral (confidence: 0.517)

✅ TASK 4 COMPLETE: Sentiment Analysis Integration


In [ ]:

print("\n" + "="*70)
print("DELIVERABLE: SAMPLE QUERY 1")
print("="*70)

query_1 = "What are customers angry about?"
print(f"\n🔍 Query: \"{query_1}\"\n")
print("🔄 Processing query through RAG pipeline...\n")


result_1 = qa_chain({"query": query_1})


print("─" * 70)
print("📝 RAG SUMMARY (Generated by LLM):")
print("─" * 70)
print(result_1['result'])
print()


print("─" * 70)
print("📚 RETRIEVED DOCUMENTS & SENTIMENT ANALYSIS:")
print("─" * 70)

if not result_1['source_documents']:
    print("❌ No relevant documents found.")
else:
  for i, doc in enumerate(result_1['source_documents'], 1):
    doc_text = doc.page_content
    doc_emotion = doc.metadata.get('emotion', 'unknown')

   
    sentiment = sentiment_pipe(doc_text)[0]
    label_name = sentiment_labels.get(sentiment['label'], sentiment['label'])

    print(f"\n📄 Document {i}:")
    print(f"   Emotion: {doc_emotion.upper()}")
    print(f"   Review: \"{doc_text}\"")
    print(f"   Sentiment: {label_name} (Score: {sentiment['score']:.3f}")
    print("   " + "─" * 66)

print("\n" + "="*70)
print("✅ Query 1 Complete")
print("="*70)


DELIVERABLE: SAMPLE QUERY 1

🔍 Query: "What are customers angry about?"

🔄 Processing query through RAG pipeline...

──────────────────────────────────────────────────────────────────────
📝 RAG SUMMARY (Generated by LLM):
──────────────────────────────────────────────────────────────────────
Rage-inducing false advertising and broken promises. Beyond angry, company refuses responsibility. Worst customer service experience ever encountered. Absolutely furious with purchase quality issues.

──────────────────────────────────────────────────────────────────────
📚 RETRIEVED DOCUMENTS & SENTIMENT ANALYSIS:
──────────────────────────────────────────────────────────────────────

📄 Document 1:
   Emotion: ANGER
   Review: "Rage-inducing false advertising and broken promises."
   Sentiment: negative (Score: 0.905
   ──────────────────────────────────────────────────────────────────

📄 Document 2:
   Emotion: ANGER
   Review: "Beyond angry, company refuses responsibility."
   Sentiment: negativ

In [ ]:

print("\n" + "="*70)
print("DELIVERABLE: SAMPLE QUERY 2")
print("="*70)

query_2 = "Tell me what the happy customers are saying"
print(f"\n🔍 Query: \"{query_2}\"\n")
print("🔄 Processing query through RAG pipeline...\n")


result_2 = qa_chain({"query": query_2})


print("─" * 70)
print("📝 RAG SUMMARY (Generated by LLM):")
print("─" * 70)
print(result_2['result'])
print()


print("─" * 70)
print("📚 RETRIEVED DOCUMENTS & SENTIMENT ANALYSIS:")
print("─" * 70)

if not result_2['source_documents']:
    print("❌ No relevant documents found.")
else:
  for i, doc in enumerate(result_2['source_documents'], 1):
    doc_text = doc.page_content
    doc_emotion = doc.metadata.get('emotion', 'unknown')


    sentiment = sentiment_pipe(doc_text)[0]
    label_name = sentiment_labels.get(sentiment['label'], sentiment['label'])

    print(f"\n📄 Document {i}:")
    print(f"   Emotion: {doc_emotion.upper()}")
    print(f"   Review: \"{doc_text}\"")
    print(f"   Sentiment: {label_name} (Score: {sentiment['score']:.3f}")
    print("   " + "─" * 66)

print("\n" + "="*70)
print("✅ Query 2 Complete")
print("="*70)


DELIVERABLE: SAMPLE QUERY 2

🔍 Query: "Tell me what the happy customers are saying"

🔄 Processing query through RAG pipeline...

──────────────────────────────────────────────────────────────────────
📝 RAG SUMMARY (Generated by LLM):
──────────────────────────────────────────────────────────────────────
Delighted by outstanding customer service experience! Happiest customer, will definitely buy again! Delighted by outstanding customer service experience!

──────────────────────────────────────────────────────────────────────
📚 RETRIEVED DOCUMENTS & SENTIMENT ANALYSIS:
──────────────────────────────────────────────────────────────────────

📄 Document 1:
   Emotion: HAPPINESS
   Review: "Delighted by outstanding customer service experience!"
   Sentiment: positive (Score: 0.984
   ──────────────────────────────────────────────────────────────────

📄 Document 2:
   Emotion: HAPPINESS
   Review: "Happiest customer, will definitely buy again!"
   Sentiment: positive (Score: 0.987
   ──────

In [ ]:

print("\n" + "="*70)
print("DELIVERABLE: SAMPLE QUERY 3 (BONUS)")
print("="*70)

query_3 = "Are there any surprised or shocked customers?"
print(f"\n🔍 Query: \"{query_3}\"\n")
print("🔄 Processing query through RAG pipeline...\n")


result_3 = qa_chain({"query": query_3})


print("─" * 70)
print("📝 RAG SUMMARY (Generated by LLM):")
print("─" * 70)
print(result_3['result'])
print()


print("─" * 70)
print("📚 RETRIEVED DOCUMENTS & SENTIMENT ANALYSIS:")
print("─" * 70)

if not result_3['source_documents']:
    print("❌ No relevant documents found.")
else:
  for i, doc in enumerate(result_3['source_documents'], 1):
    doc_text = doc.page_content
    doc_emotion = doc.metadata.get('emotion', 'unknown')

  
    sentiment = sentiment_pipe(doc_text)[0]
    label_name = sentiment_labels.get(sentiment['label'], sentiment['label'])

    print(f"\n📄 Document {i}:")
    print(f"   Emotion: {doc_emotion.upper()}")
    print(f"   Review: \"{doc_text}\"")
    print(f"   Sentiment: {label_name} (Score: {sentiment['score']:.3f}")
    print("   " + "─" * 66)

print("\n" + "="*70)
print("✅ Query 3 Complete")
print("="*70)


DELIVERABLE: SAMPLE QUERY 3 (BONUS)

🔍 Query: "Are there any surprised or shocked customers?"

🔄 Processing query through RAG pipeline...

──────────────────────────────────────────────────────────────────────
📝 RAG SUMMARY (Generated by LLM):
──────────────────────────────────────────────────────────────────────
Customer reviews: Incredible surprise beyond initial expectations! Delighted by outstanding customer service experience! Genuinely shocked by superior quality levels!

──────────────────────────────────────────────────────────────────────
📚 RETRIEVED DOCUMENTS & SENTIMENT ANALYSIS:
──────────────────────────────────────────────────────────────────────

📄 Document 1:
   Emotion: SURPRISE
   Review: "Incredible surprise beyond initial expectations!"
   Sentiment: positive (Score: 0.973
   ──────────────────────────────────────────────────────────────────

📄 Document 2:
   Emotion: HAPPINESS
   Review: "Delighted by outstanding customer service experience!"
   Sentiment: positiv

In [ ]:

print("\n" + "="*60)
print("SAVING QUERY RESULTS")
print("="*60)


all_results = [
    {'query': query_1, 'result': result_1},
    {'query': query_2, 'result': result_2},
    {'query': query_3, 'result': result_3}
]


query_results = []

for idx, item in enumerate(all_results, 1):
    query = item['query']
    result = item['result']

    for doc_idx, doc in enumerate(result['source_documents'], 1):
        sentiment = sentiment_pipe(doc.page_content)[0]
        label_name = sentiment_labels.get(sentiment['label'], sentiment['label'])

        query_results.append({
            'query_number': idx,
            'query_text': query,
            'llm_summary': result['result'],
            'retrieved_doc_number': doc_idx,
            'emotion': doc.metadata.get('emotion', 'unknown'),
            'review_text': doc.page_content,
            'sentiment_label': label_name,
            'sentiment_score': sentiment['score']
        })


query_results_path = f"{STAGE2_DIR}/query_results.csv"
results_df = pd.DataFrame(query_results)
results_df.to_csv(query_results_path, index=False)
print(f"\n💾 Query results saved to: {query_results_path}")
print(f"   Total records: {len(results_df)}")


print(f"\n📊 Results Summary:")
print(f"   Total queries: {len(all_results)}")
print(f"   Documents per query: 4")
print(f"   Total retrieved documents: {len(query_results)}")

print("\n✅ Query results saved successfully!")



SAVING QUERY RESULTS

💾 Query results saved to: /content/drive/MyDrive/AI_Assignment/stage2_outputs/query_results.csv
   Total records: 12

📊 Results Summary:
   Total queries: 3
   Documents per query: 4
   Total retrieved documents: 12

✅ Query results saved successfully!
